# Searching web with LLMs

The PageRank algorithm which is used by almost all the search engines like Google, DuckDuckGo, etc. is a search algorithm that does not involve learning. It has two main steps:

- **Matching**: Finding documents that are relevant to the given query. 
- **Ranking**: Ordering these documents from most relevant to least relevant.

This algorithm and the tasks mentioned does not generally require implementation of AI but if integrated AI could help in bringing better service and solve the problem that plagues the search algorithm. 

If an AI algorithm is implemented in search, the user can get contextualized and localized response. 

## The Process

- User writes down a query.
- Model analyzes the query and plans the action path to solve the task.
- Appropriate tools (in this case a web search) is selected.
- Documents are identified using the internet search and the information is sent back to the model.
- Model analyzes the information and decides if the goal is accomplished.
- (If still needs further breakdown) The model selects appropriate tool for further break down.
- (If accomplished) The model generates the answer and sends back to the user.

## Using LangChain

LangChain facilitates LLMs to connect with the web with the use of various tools. With the approach called **Reasoning and Acting (ReAct)** the model first considers the best strategy to get to the answer - **Reasoning**, and then it executes the strategy - **Acting**. 

LangChain offers various tools that are designed to extend the model and find information it needs. 

In the project we try to create a chain with a search engine, **DuckDuckGo**, use an external API **Serper** for Google Search, and **Wikipedia** for factual information and create a chain for a web search which gives us factual information. 

In [1]:
import os
from dotenv import load_dotenv
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

/home/sajalpaudyal/Documents/Programming_Works/web_searching_with_LLMs/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Making a chain of tools

In [2]:
ddg_search = DuckDuckGoSearchRun()
ddg_search.run('Who is the current prime minister of Nepal?')

'October 30, 2025 - The prime minister of Nepal (Nepali: नेपालको प्रधानमन्त्री, romanized: Nēpālakō pradhānamantrī) is the head of government of Federal Democratic Republic of Nepal. The prime minister leads the Council of Ministers and holds the chief executive authority in the country. 1 month ago - After the Loktantra Andolan movement ... on 20 September 2015, and the first prime minister under this new constitution was KP Sharma Oli .... October 5, 2025 - Khadga Prasad Sharma Oli (born 22 February 1952), commonly known as K. P. Sharma Oli or simply K. P. Oli, is a Nepalese politician who thrice served as the 38th prime minister of Nepal from 2015 to 2016, 2018 to 2021, and 2024 to 2025. 1 week ago - Upon her recommendation, president Ram Chandra Poudel dissolved the Federal Parliament of Nepal on 12 September, and Karki was sworn in as interim Prime Minister based on article 61 of the Constitution of Nepal. Nepal ,[a] officially the Federal Democratic Republic of Nepal ,[b] is a la

In [3]:
@tool("ddg_search")
def search_ddg(search_query:str) -> str:
    
    """
    A web search tool to extract information from the internet.
    
    Args:
        search_query(str): The query provided by the user to search for.
    """
    return ddg_search.run(search_query)

In [4]:
load_dotenv()

SERPER_KEY = os.getenv("SERPER_API_KEY")

In [5]:
google_search = GoogleSerperAPIWrapper()

@tool("google_search")
def search_google(search_query:str) -> str:
    """
    A web search tool to extract information from the internet (Using Google Search).
    
    Args:
        Search Query(str): The query provided by the user to search for.
    """
    return google_search.run(search_query)

In [6]:


wiki_search =  WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=None))

@tool("wiki_search")
def search_wiki(search_query:str) -> str:
    """
    A web search tool to extract information from the internet (Using Wikipedia).
    
    Args:
        Search Query(str): The query provided by the user to search for.
    """
    
    return wiki_search.run(search_query)

In [7]:
model_name = "microsoft/Phi-3-mini-128k-instruct"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:  50%|█████     | 1/2 [00:08<00:08,  8.37s/it]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
pipe = pipeline("text-generation", model = model, tokenizer = tokenizer, max_new_tokens = 500, temperature = 0.1, top_k=50)

Device set to use cpu


In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
chat_model = ChatHuggingFace(llm=llm)

In [ ]:
agent = create_agent(
    model=chat_model,
    tools = [search_ddg, search_google, search_wiki],
)

In [ ]:
query = "How did the Napoleonic war end?"

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": query}]}
)
result

{'messages': [HumanMessage(content='How did the Napoleonic war end?', additional_kwargs={}, response_metadata={}, id='f0e6f18f-6e07-4322-90cb-854b4dfa145b'),
  AIMessage(content="<|user|>\nHow did the Napoleonic war end?<|end|>\n<|assistant|>\n The Napoleonic Wars, a series of conflicts involving Napoleon's French Empire and changing sets of European allies, ended with Napoleon Bonaparte's defeat at the Battle of Waterloo on June 18, 1815. This defeat led to Napoleon's abdication and exile to the island of Saint Helena, where he died in 1821. The Congress of Vienna, which had been interrupted by Napoleon's return from exile in 1815, resumed its work and established a new balance of power in Europe, aiming to prevent the rise of a single dominant power and maintain peace.", additional_kwargs={}, response_metadata={}, id='lc_run--019af010-4ca6-7cb2-a1ad-e0bdecbef63b-0')]}

In [ ]:
print("-" * 30)
print(result["messages"][-1].content)

------------------------------
<|user|>
How did the Napoleonic war end?<|end|>
<|assistant|>
 The Napoleonic Wars, a series of conflicts involving Napoleon's French Empire and changing sets of European allies, ended with Napoleon Bonaparte's defeat at the Battle of Waterloo on June 18, 1815. This defeat led to Napoleon's abdication and exile to the island of Saint Helena, where he died in 1821. The Congress of Vienna, which had been interrupted by Napoleon's return from exile in 1815, resumed its work and established a new balance of power in Europe, aiming to prevent the rise of a single dominant power and maintain peace.
